# Machine Learning per Analisi Blockchain

Questo notebook dimostra l'uso dei modelli ML per analizzare transazioni Bitcoin.

In [ ]:
import sys
sys.path.append('../ml')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from train_models import (
    load_transaction_data, engineer_features,
    CoinJoinDetector, FeeratePredictor,
    TransactionClusterer, AnomalyDetector
)

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

## 1. Caricamento e Preparazione Dati

In [ ]:
# Carica dati dal database
df = load_transaction_data(min_samples=500)
print(f"Caricate {len(df)} transazioni")

# Feature engineering
df = engineer_features(df)

# Mostra prime righe
df.head()

In [ ]:
# Statistiche descrittive
df[['feerate', 'inputs_count', 'outputs_count', 'vsize', 'coinjoin_score']].describe()

## 2. Analisi Esplorativa

In [ ]:
# Distribuzione feerate
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Feerate histogram
axes[0, 0].hist(df['feerate'], bins=50, edgecolor='black')
axes[0, 0].set_xlabel('Feerate (sat/vB)')
axes[0, 0].set_ylabel('Frequenza')
axes[0, 0].set_title('Distribuzione Feerate')

# Inputs vs Outputs
axes[0, 1].scatter(df['inputs_count'], df['outputs_count'], alpha=0.5)
axes[0, 1].set_xlabel('Inputs')
axes[0, 1].set_ylabel('Outputs')
axes[0, 1].set_title('Inputs vs Outputs')

# CoinJoin Score distribution
axes[1, 0].hist(df['coinjoin_score'], bins=30, edgecolor='black')
axes[1, 0].set_xlabel('CoinJoin Score')
axes[1, 0].set_ylabel('Frequenza')
axes[1, 0].set_title('Distribuzione CoinJoin Score')

# RBF vs non-RBF feerate
df.boxplot(column='feerate', by='is_rbf', ax=axes[1, 1])
axes[1, 1].set_xlabel('RBF Abilitato')
axes[1, 1].set_ylabel('Feerate (sat/vB)')
axes[1, 1].set_title('Feerate: RBF vs non-RBF')

plt.tight_layout()
plt.show()

## 3. Training e Valutazione Modelli

### 3.1 CoinJoin Detector

In [ ]:
# Training CoinJoin Detector
coinjoin_detector = CoinJoinDetector()
coinjoin_detector.train(df)
coinjoin_detector.save()

# Predizioni
df['coinjoin_probability'] = coinjoin_detector.predict(df)

In [ ]:
# Visualizza distribuzione probabilità CoinJoin
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.hist(df['coinjoin_probability'], bins=50, edgecolor='black')
plt.xlabel('Probabilità CoinJoin')
plt.ylabel('Frequenza')
plt.title('Distribuzione Probabilità CoinJoin')

plt.subplot(1, 2, 2)
high_prob = df[df['coinjoin_probability'] > 0.7]
plt.scatter(high_prob['inputs_count'], high_prob['outputs_count'], 
           c=high_prob['coinjoin_probability'], cmap='Reds', s=100, alpha=0.6)
plt.colorbar(label='Probabilità CoinJoin')
plt.xlabel('Inputs')
plt.ylabel('Outputs')
plt.title('CoinJoin Candidates (prob > 0.7)')

plt.tight_layout()
plt.show()

print(f"\nTransazioni con alta probabilità CoinJoin (>0.7): {len(high_prob)}")

### 3.2 Feerate Predictor

In [ ]:
# Training Feerate Predictor
feerate_predictor = FeeratePredictor()
feerate_predictor.train(df)
feerate_predictor.save()

# Predizioni
df['predicted_feerate'] = feerate_predictor.predict(df)
df['feerate_error'] = df['feerate'] - df['predicted_feerate']

In [ ]:
# Visualizza predizioni vs realtà
plt.figure(figsize=(15, 5))

# Scatter plot: predetto vs reale
plt.subplot(1, 3, 1)
plt.scatter(df['feerate'], df['predicted_feerate'], alpha=0.5)
max_val = max(df['feerate'].max(), df['predicted_feerate'].max())
plt.plot([0, max_val], [0, max_val], 'r--', label='Perfect prediction')
plt.xlabel('Feerate Reale (sat/vB)')
plt.ylabel('Feerate Predetto (sat/vB)')
plt.title('Predetto vs Reale')
plt.legend()

# Distribuzione errori
plt.subplot(1, 3, 2)
plt.hist(df['feerate_error'], bins=50, edgecolor='black')
plt.xlabel('Errore (Reale - Predetto)')
plt.ylabel('Frequenza')
plt.title('Distribuzione Errori di Predizione')
plt.axvline(0, color='red', linestyle='--', label='Zero error')
plt.legend()

# Errore nel tempo (per block height)
plt.subplot(1, 3, 3)
df_sorted = df.sort_values('block_height')
plt.plot(df_sorted['block_height'], df_sorted['feerate_error'].rolling(100).mean())
plt.xlabel('Block Height')
plt.ylabel('Errore Medio (rolling 100)')
plt.title('Errore di Predizione nel Tempo')
plt.axhline(0, color='red', linestyle='--')

plt.tight_layout()
plt.show()

### 3.3 Transaction Clustering

In [ ]:
# Clustering
clusterer = TransactionClusterer(method='kmeans', n_clusters=5)
clusters = clusterer.fit(df)
clusterer.save()

df['cluster'] = clusters

In [ ]:
# Visualizza clusters
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Cluster distribution
df['cluster'].value_counts().sort_index().plot(kind='bar', ax=axes[0, 0])
axes[0, 0].set_xlabel('Cluster ID')
axes[0, 0].set_ylabel('Numero Transazioni')
axes[0, 0].set_title('Distribuzione Clusters')

# Scatter: feerate vs vsize colorato per cluster
for cluster_id in sorted(df['cluster'].unique()):
    cluster_data = df[df['cluster'] == cluster_id]
    axes[0, 1].scatter(cluster_data['vsize'], cluster_data['feerate'], 
                      label=f'Cluster {cluster_id}', alpha=0.6)
axes[0, 1].set_xlabel('vSize (bytes)')
axes[0, 1].set_ylabel('Feerate (sat/vB)')
axes[0, 1].set_title('Clusters: vSize vs Feerate')
axes[0, 1].legend()

# Boxplot feerate per cluster
df.boxplot(column='feerate', by='cluster', ax=axes[1, 0])
axes[1, 0].set_xlabel('Cluster')
axes[1, 0].set_ylabel('Feerate (sat/vB)')
axes[1, 0].set_title('Feerate per Cluster')

# Heatmap caratteristiche medie per cluster
cluster_features = df.groupby('cluster')[[
    'feerate', 'vsize', 'inputs_count', 'outputs_count', 'coinjoin_score'
]].mean()
sns.heatmap(cluster_features.T, annot=True, fmt='.2f', cmap='YlOrRd', ax=axes[1, 1])
axes[1, 1].set_title('Caratteristiche Medie per Cluster')

plt.tight_layout()
plt.show()

In [ ]:
# Profilo dettagliato di ogni cluster
print("PROFILO CLUSTERS:\n" + "="*60)
for cluster_id in sorted(df['cluster'].unique()):
    cluster_data = df[df['cluster'] == cluster_id]
    print(f"\n📊 CLUSTER {cluster_id} ({len(cluster_data)} transazioni, {len(cluster_data)/len(df):.1%})")
    print(f"  Feerate medio: {cluster_data['feerate'].mean():.2f} sat/vB")
    print(f"  vSize medio: {cluster_data['vsize'].mean():.0f} bytes")
    print(f"  Input medio: {cluster_data['inputs_count'].mean():.1f}")
    print(f"  Output medio: {cluster_data['outputs_count'].mean():.1f}")
    print(f"  RBF rate: {cluster_data['is_rbf'].mean():.1%}")
    print(f"  CoinJoin score: {cluster_data['coinjoin_score'].mean():.3f}")

### 3.4 Anomaly Detection

In [ ]:
# Anomaly detection
anomaly_detector = AnomalyDetector(contamination=0.05)
anomalies = anomaly_detector.fit(df)
anomaly_detector.save()

df['is_anomaly'] = anomalies == -1

In [ ]:
# Visualizza anomalie
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Scatter plot con anomalie evidenziate
normal = df[~df['is_anomaly']]
anomalous = df[df['is_anomaly']]

axes[0].scatter(normal['vsize'], normal['feerate'], alpha=0.5, label='Normale', s=20)
axes[0].scatter(anomalous['vsize'], anomalous['feerate'], 
               color='red', alpha=0.8, label='Anomalo', s=100, marker='x')
axes[0].set_xlabel('vSize (bytes)')
axes[0].set_ylabel('Feerate (sat/vB)')
axes[0].set_title('Anomalie: vSize vs Feerate')
axes[0].legend()

# Scatter: inputs vs outputs
axes[1].scatter(normal['inputs_count'], normal['outputs_count'], 
               alpha=0.5, label='Normale', s=20)
axes[1].scatter(anomalous['inputs_count'], anomalous['outputs_count'], 
               color='red', alpha=0.8, label='Anomalo', s=100, marker='x')
axes[1].set_xlabel('Inputs')
axes[1].set_ylabel('Outputs')
axes[1].set_title('Anomalie: Inputs vs Outputs')
axes[1].legend()

plt.tight_layout()
plt.show()

print(f"\n🚨 Anomalie rilevate: {df['is_anomaly'].sum()} ({df['is_anomaly'].mean():.1%})")

In [ ]:
# Top 10 anomalie più interessanti
print("\nTOP 10 ANOMALIE:\n" + "="*80)
anomaly_df = df[df['is_anomaly']].nlargest(10, 'feerate')

for idx, row in anomaly_df.iterrows():
    print(f"\nTXID: {row['txid'][:32]}...")
    print(f"  Block: {row['block_height']:.0f}")
    print(f"  Feerate: {row['feerate']:.2f} sat/vB")
    print(f"  Fee: {row['fee']:.0f} sat")
    print(f"  Size: {row['vsize']:.0f} vB")
    print(f"  I/O: {row['inputs_count']:.0f}/{row['outputs_count']:.0f}")
    print(f"  RBF: {'Sì' if row['is_rbf'] else 'No'}")
    print(f"  CoinJoin prob: {row.get('coinjoin_probability', 0):.1%}")

## 4. Analisi Comparative

In [ ]:
# Confronto tra diverse categorie
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# CoinJoin vs Non-CoinJoin
high_coinjoin = df['coinjoin_probability'] > 0.7
df['category'] = 'Normal'
df.loc[high_coinjoin, 'category'] = 'CoinJoin'
df.loc[df['is_anomaly'], 'category'] = 'Anomaly'

# Feerate comparison
df.boxplot(column='feerate', by='category', ax=axes[0, 0])
axes[0, 0].set_xlabel('Categoria')
axes[0, 0].set_ylabel('Feerate (sat/vB)')
axes[0, 0].set_title('Feerate per Categoria')

# Size comparison
df.boxplot(column='vsize', by='category', ax=axes[0, 1])
axes[0, 1].set_xlabel('Categoria')
axes[0, 1].set_ylabel('vSize (bytes)')
axes[0, 1].set_title('Size per Categoria')

# Input count comparison
df.boxplot(column='inputs_count', by='category', ax=axes[1, 0])
axes[1, 0].set_xlabel('Categoria')
axes[1, 0].set_ylabel('Inputs')
axes[1, 0].set_title('Inputs per Categoria')

# Output count comparison
df.boxplot(column='outputs_count', by='category', ax=axes[1, 1])
axes[1, 1].set_xlabel('Categoria')
axes[1, 1].set_ylabel('Outputs')
axes[1, 1].set_title('Outputs per Categoria')

plt.tight_layout()
plt.show()

## 5. Predizioni su Nuove Transazioni

In [ ]:
# Simula nuove transazioni
import datetime

now = datetime.datetime.now()

new_txs = pd.DataFrame([
    # Transazione normale
    {
        'inputs_count': 2,
        'outputs_count': 2,
        'vsize': 250,
        'size': 250,
        'weight': 1000,
        'hour': now.hour,
        'day_of_week': now.weekday(),
        'is_weekend': 1 if now.weekday() >= 5 else 0,
        'is_rbf_int': 1,
        'coinjoin_score': 0.0,
        'equal_output': False
    },
    # Possibile CoinJoin
    {
        'inputs_count': 5,
        'outputs_count': 5,
        'vsize': 800,
        'size': 850,
        'weight': 3200,
        'hour': now.hour,
        'day_of_week': now.weekday(),
        'is_weekend': 1 if now.weekday() >= 5 else 0,
        'is_rbf_int': 1,
        'coinjoin_score': 0.8,
        'equal_output': True
    },
    # Transazione grande
    {
        'inputs_count': 10,
        'outputs_count': 3,
        'vsize': 1500,
        'size': 1600,
        'weight': 6000,
        'hour': now.hour,
        'day_of_week': now.weekday(),
        'is_weekend': 1 if now.weekday() >= 5 else 0,
        'is_rbf_int': 0,
        'coinjoin_score': 0.0,
        'equal_output': False
    }
])

# Calcola features derivate
new_txs['io_ratio'] = new_txs['inputs_count'] / new_txs['outputs_count']
new_txs['weight_to_size_ratio'] = new_txs['weight'] / new_txs['size']
new_txs['fee_per_input'] = 0  # Placeholder
new_txs['fee_per_output'] = 0  # Placeholder
new_txs['size_per_input'] = new_txs['vsize'] / new_txs['inputs_count']
new_txs['size_per_output'] = new_txs['vsize'] / new_txs['outputs_count']

print("PREDIZIONI PER NUOVE TRANSAZIONI:\n" + "="*80)

for idx, tx in new_txs.iterrows():
    print(f"\n{'='*80}")
    print(f"TRANSAZIONE #{idx + 1}")
    print(f"  Inputs: {tx['inputs_count']:.0f}, Outputs: {tx['outputs_count']:.0f}")
    print(f"  vSize: {tx['vsize']:.0f} vB")
    
    # Predici feerate
    predicted_feerate = feerate_predictor.predict(pd.DataFrame([tx]))[0]
    print(f"\n  💰 Feerate raccomandato: {predicted_feerate:.2f} sat/vB")
    print(f"  💸 Fee stimata: {predicted_feerate * tx['vsize']:.0f} sat")
    
    # Predici CoinJoin
    coinjoin_prob = coinjoin_detector.predict(pd.DataFrame([tx]))[0]
    print(f"\n  🔀 Probabilità CoinJoin: {coinjoin_prob:.1%}")
    if coinjoin_prob > 0.7:
        print(f"     ⚠️  ALTA probabilità di CoinJoin!")
    
    # Cluster assignment
    X = clusterer.prepare_data(pd.DataFrame([tx]))
    X_scaled = clusterer.scaler.transform(X)
    cluster_id = clusterer.model.predict(X_scaled)[0]
    print(f"\n  📊 Cluster assegnato: {cluster_id}")

## 6. Salvataggio Risultati

In [ ]:
# Salva risultati completi
output_file = '../ml/results/ml_analysis_complete.csv'
df[[
    'txid', 'block_height', 'feerate', 'predicted_feerate',
    'coinjoin_probability', 'cluster', 'is_anomaly', 'category'
]].to_csv(output_file, index=False)

print(f"✅ Risultati salvati in: {output_file}")

# Summary finale
print("\n" + "="*80)
print("SUMMARY FINALE")
print("="*80)
print(f"Totale transazioni analizzate: {len(df)}")
print(f"CoinJoin candidates (prob > 0.7): {(df['coinjoin_probability'] > 0.7).sum()}")
print(f"Anomalie rilevate: {df['is_anomaly'].sum()}")
print(f"Clusters identificati: {df['cluster'].nunique()}")
print(f"\nFeerate medio: {df['feerate'].mean():.2f} sat/vB")
print(f"Feerate mediano: {df['feerate'].median():.2f} sat/vB")
print(f"RBF adoption rate: {df['is_rbf'].mean():.1%}")